In [ ]:
# Data imputation methods.

# We consider two methods of imputation of data, which might be helpful.

# 1) Model-based imputation

# 2) Imputation using k-means clustering

#NOTE: Imputation with the mean or by dropping observations with missing values 
# are trivial with Pandas and can be googled very quickly!

In [ ]:
#1) Impute with other predictors for a given observation

def impute_missing_model(non_nan_df,nan_df,response_col_name):
    #return a dataset with missing values imputed by other predictors. In process, you have to drop the name of the response col

    #build model from dataset without missing vals
    cols_missing_values = nan_df.columns[nan_df.isnull().any()]

    X_imp = non_nan_df.drop(cols_missing_values,axis=1).drop(response_col_name,axis=1)
    mod_dataset_imp = nan_df.copy()

    for item in cols_missing_values:
        y_imp = non_nan_df[item]
        #print("imputing",item)
        #create regression model and fit in predictors, the predicted value will be the column with NaNs
        regress = LinearRegression()
        regress.fit(X_imp,y_imp)
        y_hat = regress.predict(X_imp)

    #Now find the rows in the whole dataframe with missing values (X_missing stores the features without missing values)
        X_missing = nan_df[nan_df[item].isnull()].drop(cols_missing_values,axis=1).drop(response_col_name,axis=1)

    #predict missing values based on the features without missing values
        y_missing = regress.predict(X_missing)
        y_missing_noise = y_missing+np.random.normal(loc=0,scale=np.sqrt(mean_squared_error(y_imp,y_hat)),size=y_missing.shape[0])

    #get the indices of missing values
        missing_index = nan_df[item][nan_df[item].isnull()].index
        missing_series = pd.Series(data = y_missing_noise, index = missing_index)
        mod_dataset_imp[item] = mod_dataset_imp[item].fillna(missing_series)
    
    return mod_dataset_imp

In [ ]:
# Method to impute missing values using k-means clustering
    def kmeans_missing(X, n_clusters, max_iter=10):

        # Initialize missing values to their column means
        missing = ~np.isfinite(X)
        mu = np.nanmean(X, 0, keepdims=1)
        X_hat = np.where(missing, mu, X)

        for i in range(max_iter):
            if i > 0:
                cls = KMeans(n_clusters, init=prev_centroids)
            else:
                cls = KMeans(n_clusters, n_jobs=-1)

            # Perform clustering on the filled-in data
            labels = cls.fit_predict(X_hat)
            centroids = cls.cluster_centers_

            # Fill in the missing values based on their cluster centroids
            X_hat[missing] = centroids[labels][missing]

            # When the labels have stopped changing then we have converged
            if i > 0 and np.all(labels == prev_labels):
                break

            prev_labels = labels
            prev_centroids = cls.cluster_centers_

        return X_hat